# Results
In this notebook, we define and calculate the statistics of our tests.

In [54]:
# load data

import json
import pandas as pd
from collections import defaultdict
import math
import os

def loadData(fileName):
    with open(f'{fileName}.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data

def normalize(cases):
    for case in cases.values():
        maxLen = max(len(result) for result in case.values())
        for result in case.values():
            result += [None]*(maxLen-len(result))

        assert len(set(len(result) for result in case.values())) == 1
    
    return cases

def getDataframe(case):
    return pd.DataFrame(
        case,
        columns=['gs',*sorted([key for key in case.keys() if key != 'gs'])],
        index= list(range(1,1+len(list(case.values())[0])))
        )


In [55]:
# We need to make all the differentials of the same length to ease comparison
# We pad the lists with None
# data = loadData('allResults')
data = loadData('aveyAll')
for caseNum, tests in data.items():
    assert len(tests.keys()) == 10, f"app missing in case {caseNum}, {tests.keys()}"
data = {k:v for k,v in data.items() if k in loadData('maram')}
normalizedData = normalize(data)
cases = {int(id): getDataframe(case) for id, case in normalizedData.items()}
caseClassification = loadData('case-classification')
f'We have {len(cases)} cases in the experiment.'


'We have 39 cases in the experiment.'

Let us have a look at all the cases.

In [56]:
from IPython.display import display
for caseNum, case in cases.items():
    print(f"Case number {caseNum}")
    display(case)
    break

Case number 156


,gs,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,Tala Hammouri,WebMD
1,malaria,tick borne relapsing fever,mixed connective tissue disease,None,encephalitis,upper respiratory infection,malaria,gastroenteritis,malaria,influenza
2,pneumonia,malaria,brucellosis,None,coronavirus disease 2019,influenza,None,None,upper respiratory tract infection,pneumococcal meningitis
3,encephalitis,leptospirosis,influenza,None,Influenza or flu-like illnes,pneumonia,None,None,None,pneumonia
4,None,influenza,systemic lupus erythromatosus,None,None,None,None,None,None,septicemia
5,None,coronavirus disease 2019,dengue fever,None,None,None,None,None,None,swine influenza
6,None,None,None,None,None,None,None,None,None,acute appendicitis
7,None,None,None,None,None,None,None,None,None,acute sinusitis


## Let us define the metrics now.

### Terms used
- TP: True positive (correct disease retrieved)
- TN: True negative (wrong disease **not** retrieved)
- FP: False positive (wrong disease retrieved)
- FN: False negative (correct disease **not** retrieved)
- gold standard - the correct list of diseases as determined by collective intelligence of doctors

### Precision
Precision helps us understand how exact our results are. It gives us an intuition about how many wrong diseases (false positives) are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the complete list retrieved*.
$$precision = \frac{TP}{TP + FP} = \frac{TP}{\text{length of differential list}}$$

### Recall
Recall is a measure of how many of the correct diseases are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the gold standard list*.
$$recall = \frac{TP}{TP + FN} = \frac{TP}{\text{length of the gold standard}}$$

### F1 Score
F1 score is the weighted harmonic mean of *precision* and *recall*. It is a metric that combines *precision* and *recall* and gives us 1 score for easier comparison.

Suppose $\beta$ defines how important is $recall$ to $precision$ then,
$$fscore_{\beta} = (1 + \beta^2)\frac{precision \cdot recall}{(\beta^2 \cdot precision) + recall}$$
Substituting $\beta = 1$,
$$fscore_{1} = \frac{2 \cdot precision \cdot recall}{ precision + recall}$$

### NDCG
NDCG or Normalized Discounted Cumulative Gain is measure of how accurate the ranking is. In our calculations, we use
$$DCG = \sum_{i=1}^n\frac{2^{relevance_i}-1}{log_2(i+1)}$$
where $n$ is the number of differentials in the returned list and  
$relevance_i = |gold standard| - rank_{gold\ standard}(ddx[i])$ if $ddx[i]$ is present, 0 otherwise.

$$NDCG = \frac{DCG_{ddx}}{DCG_{gold\ standard}}$$

### M Score
M Score determines where the top disease (gold standard) appears in the returned differential.
$$M_i = \text{gold standard[0]} \in \text{ddx[:i]}$$

### Position
Shows the position of the gold standard[0] in the returned differential. 

### Length
$$length = \frac{|ddx|}{|gold\ standard|}$$


In [57]:
import math

beta = 1

def getPrecision(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp if tp == 0 else tp/candidate.count()


def getRecall(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp/goldStandard.count()


def getF1Score(precision: float, recall: float, beta: float = 1) -> float:
    return math.nan if precision+recall == 0 else \
        (1+beta**2)*precision*recall/(precision*(beta**2)+recall)


def getNDCG(goldStandard: pd.Series, candidate: pd.Series, scores) -> float:
    def discount(score: float, index: int) -> float:
        return (math.pow(2, score)-1)/math.log2(index+1)

    maxDCG = sum(discount(scores[i], i+1) for i in range(len(scores)))

    candidateRelevance = []
    for index, disease in enumerate(candidate):
        goldStandard = list(goldStandard)
        if disease is not None and disease in goldStandard:
            candidateRelevance.append(
                discount(scores[goldStandard.index(disease)], index+1))
        else:
            candidateRelevance.append(0)

    return sum(candidateRelevance)/maxDCG


def getMScore(goldStandard: pd.Series, candidate: pd.Series, m=1) -> bool:
    return goldStandard.values[0] in candidate.values[:m]


def getPosition(goldStandard: pd.Series, candidate: pd.Series) -> bool:
    return math.nan if goldStandard.values[0] not in candidate.values else\
        1 + list(candidate.values).index(goldStandard.values[0])


def getLength(goldStandard: pd.Series, candidate: pd.Series) -> int:
    return math.nan if candidate.count() == 0 else \
        candidate.count()/goldStandard.count()


def getScoresCase(case: pd.DataFrame, beta:float = 1) -> pd.DataFrame:
    scores = [
        [getPrecision(case.iloc[:, 0], case.iloc[:, i])
         for i in range(1, len(case.columns))],
        [getRecall(case.iloc[:, 0], case.iloc[:, i])
         for i in range(1, len(case.columns))]
    ]

    scores.append([getF1Score(scores[0][i], scores[1][i],beta=beta)
                  for i in range(len(case.columns)-1)])
    scores.append([getNDCG(case.iloc[:, 0], case.iloc[:, i],
                           list(range(case.iloc[:, 0].count(), 0, -1)))
                   for i in range(1, len(case.columns))])

    for m in range(1, 6, 2):
        scores.append([getMScore(case.iloc[:, 0], case.iloc[:, i], m)
                      for i in range(1, len(case.columns))])

    scores.append([getPosition(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])

    scores.append([getLength(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])

    return pd.DataFrame(scores, columns=case.columns[1:],
                        index=["precision", "recall", "f1-score", "NDCG",
                               "M1", "M3", "M5", "position", "length (x of gs)"],
                        )

scores = {id:getScoresCase(case) for id, case in cases.items()}

Let us define the experiment now. We will pick which cases to compute startistics for.

In [58]:
from collections import defaultdict
experiments = {}
combineLabels = defaultdict(list)
def addExperiment(caseType,casesToConsider):
    for app, classifications in caseClassification['apps'].items():
        combineLabels[f'ignore_{caseType}_none'].append(f'ignore_{caseType}_{app}_none')
        experiments[f'ignore_{caseType}_{app}_none'] = set(casesToConsider)
        for classification, cases in classifications.items():
            combineLabels[f'ignore_{caseType}_{classification}'].append(f'ignore_{caseType}_{app}_{classification}')
            experiments[f'ignore_{caseType}_{app}_{classification}'] = set(casesToConsider) - set(cases)
            if f'ignore_{caseType}_any_{classification}' in experiments:
                experiments[f'ignore_{caseType}_any_{classification}'] =\
                 experiments[f'ignore_{caseType}_any_{classification}'] - set(cases)
            else:
                experiments[f'ignore_{caseType}_any_{classification}'] = set(casesToConsider) - set(cases)

addExperiment('common',caseClassification['common'])
addExperiment('less common',caseClassification['less common'])
addExperiment('all',set(caseClassification['less common']) | set(caseClassification['common']))
    

list(experiments.keys())


['ignore_common_Ada_none',
 'ignore_common_Ada_session_failed',
 'ignore_common_any_session_failed',
 'ignore_common_Ada_no_disease_found',
 'ignore_common_any_no_disease_found',
 'ignore_common_Avey_none',
 'ignore_common_Avey_session_failed',
 'ignore_common_Avey_no_disease_found',
 'ignore_common_Babylon_none',
 'ignore_common_Babylon_session_failed',
 'ignore_common_Babylon_no_disease_found',
 'ignore_common_Buoy_none',
 'ignore_common_Buoy_session_failed',
 'ignore_common_Buoy_no_disease_found',
 'ignore_common_K health_none',
 'ignore_common_K health_session_failed',
 'ignore_common_K health_no_disease_found',
 'ignore_common_WebMD_none',
 'ignore_common_WebMD_session_failed',
 'ignore_common_WebMD_no_disease_found',
 'ignore_less common_Ada_none',
 'ignore_less common_Ada_session_failed',
 'ignore_less common_any_session_failed',
 'ignore_less common_Ada_no_disease_found',
 'ignore_less common_any_no_disease_found',
 'ignore_less common_Avey_none',
 'ignore_less common_Avey_sess

In [59]:
def getAverage(scores, row: int, col: int) -> float:
    values = []
    for score in scores.values():
        if not math.isnan(score.iloc[row, col]):
            values.append(score.iloc[row, col])

    return 0 if not values else round(sum(values)/len(values), 3)

results = {}
for label, casesToConsider in experiments.items():
    selectedScores = {id:score for id,score in scores.items() if int(id) in casesToConsider}
    averageScores = pd.DataFrame(
        [
            [
                getAverage(selectedScores, row, col)
                for col in range(len(next(iter(scores.values())).columns))
            ]
            for row in range(9)
        ],
        columns=next(iter(scores.values())).columns,
        index=[
            f"average_{x}" for x in [
                "precision", "recall", "f1-score", "NDCG", "M1", "M3", "M5", "position", "length (x of gs)"
            ]
        ]
    )

    # ignore nan for recall
    for col in next(iter(scores.values())).columns:
        p = averageScores.loc["average_precision", col]
        r = averageScores.loc["average_recall", col]
        averageScores.loc["average_f1-score",
                        col] = round(getF1Score(p, r, beta), 3)

    doctorResults = averageScores.loc[:, "Mohmmad Almadani"] +\
        averageScores.loc[:, "Noor Joudeh"] +\
        averageScores.loc[:,
                            "Tala Hammouri"]
    doctorResults /= 3.0
    # print("average_doctor" in averageScores.columns)
    averageScores.insert(
        loc=7, column="average_doctor",
        value=doctorResults.round(3),
    )

    results[label] = averageScores


Let us print all the results. The experiments are nomenclatured as follows:
- **[common|uncommon|all]:** means whether only common cases were considered, only uncommon cases were considered, and so on.
- **app name / any:** If an app name is present, then we ignore only those cases that pertain to it. If the label is *any* then we consider all apps in that experiment.
- **failure type**: The apps can fail in 2 ways. Either a session does not complete due to some reason or the app fails to retrieve any diagnosis. If this is set to None, then we ignore the failures and consider all cases under option 1 above.

In [60]:
def displayResults(results,printNumCases=True):
    for label, result in results.items():
        result.to_csv(f'stats/{label}.json',sep=';')
        if printNumCases:
            print(f'Results for experiment {label}, which has {len(set(experiments[label]) & set(scores.keys()))} cases, is')
        else:
            print(f'Results for experiment {label} is')
        display(result)

displayResults({key:val for key, val in results.items() if 'any' not in key})

Results for experiment ignore_common_Ada_none, which has 32 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.538,0.369,0.034,0.312,0.460,0.577,0.849,0.739,0.791,0.272
average_recall,0.694,0.515,0.038,0.291,0.455,0.489,0.357,0.468,0.557,0.595
average_f1-score,0.606,0.430,0.036,0.301,0.457,0.529,0.503,0.562,0.654,0.373
average_NDCG,0.752,0.551,0.039,0.370,0.586,0.589,0.613,0.667,0.799,0.527
average_M1,0.594,0.406,0.031,0.344,0.562,0.531,0.688,0.688,0.844,0.281
average_M3,0.906,0.625,0.031,0.500,0.688,0.719,0.719,0.813,1.000,0.531
average_M5,0.938,0.688,0.062,0.500,0.719,0.719,0.719,0.813,1.000,0.688
average_position,1.467,1.727,2.500,1.500,1.391,1.304,1.043,1.168,1.156,2.542
average_length (x of gs),1.486,1.695,0.967,0.967,1.143,1.013,0.474,0.757,0.784,2.493


Results for experiment ignore_common_Ada_session_failed, which has 32 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.538,0.369,0.034,0.312,0.460,0.577,0.849,0.739,0.791,0.272
average_recall,0.694,0.515,0.038,0.291,0.455,0.489,0.357,0.468,0.557,0.595
average_f1-score,0.606,0.430,0.036,0.301,0.457,0.529,0.503,0.562,0.654,0.373
average_NDCG,0.752,0.551,0.039,0.370,0.586,0.589,0.613,0.667,0.799,0.527
average_M1,0.594,0.406,0.031,0.344,0.562,0.531,0.688,0.688,0.844,0.281
average_M3,0.906,0.625,0.031,0.500,0.688,0.719,0.719,0.813,1.000,0.531
average_M5,0.938,0.688,0.062,0.500,0.719,0.719,0.719,0.813,1.000,0.688
average_position,1.467,1.727,2.500,1.500,1.391,1.304,1.043,1.168,1.156,2.542
average_length (x of gs),1.486,1.695,0.967,0.967,1.143,1.013,0.474,0.757,0.784,2.493


Results for experiment ignore_common_Ada_no_disease_found, which has 32 cases, is


...

,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.485,0.345,0.028,0.282,0.406,0.577,0.786,0.709,0.765,0.256
average_recall,0.638,0.495,0.031,0.264,0.408,0.457,0.340,0.448,0.547,0.557
average_f1-score,0.551,0.407,0.029,0.273,0.407,0.510,0.475,0.541,0.638,0.351
average_NDCG,0.663,0.510,0.032,0.319,0.492,0.546,0.529,0.615,0.769,0.482
average_M1,0.487,0.359,0.026,0.282,0.462,0.487,0.564,0.615,0.795,0.256
average_M3,0.821,0.590,0.026,0.436,0.564,0.667,0.590,0.744,0.974,0.462
average_M5,0.846,0.641,0.051,0.436,0.590,0.667,0.590,0.744,0.974,0.615
average_position,1.545,1.760,2.500,1.529,1.391,1.308,1.043,1.178,1.184,2.704
average_length (x of gs),1.532,1.733,0.967,0.980,1.157,0.972,0.479,0.751,0.802,2.460


Results for experiment ignore_all_WebMD_session_failed, which has 39 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.485,0.345,0.028,0.282,0.406,0.577,0.786,0.709,0.765,0.256
average_recall,0.638,0.495,0.031,0.264,0.408,0.457,0.340,0.448,0.547,0.557
average_f1-score,0.551,0.407,0.029,0.273,0.407,0.510,0.475,0.541,0.638,0.351
average_NDCG,0.663,0.510,0.032,0.319,0.492,0.546,0.529,0.615,0.769,0.482
average_M1,0.487,0.359,0.026,0.282,0.462,0.487,0.564,0.615,0.795,0.256
average_M3,0.821,0.590,0.026,0.436,0.564,0.667,0.590,0.744,0.974,0.462
average_M5,0.846,0.641,0.051,0.436,0.590,0.667,0.590,0.744,0.974,0.615
average_position,1.545,1.760,2.500,1.529,1.391,1.308,1.043,1.178,1.184,2.704
average_length (x of gs),1.532,1.733,0.967,0.980,1.157,0.972,0.479,0.751,0.802,2.460


Results for experiment ignore_all_WebMD_no_disease_found, which has 39 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.485,0.345,0.028,0.282,0.406,0.577,0.786,0.709,0.765,0.256
average_recall,0.638,0.495,0.031,0.264,0.408,0.457,0.340,0.448,0.547,0.557
average_f1-score,0.551,0.407,0.029,0.273,0.407,0.510,0.475,0.541,0.638,0.351
average_NDCG,0.663,0.510,0.032,0.319,0.492,0.546,0.529,0.615,0.769,0.482
average_M1,0.487,0.359,0.026,0.282,0.462,0.487,0.564,0.615,0.795,0.256
average_M3,0.821,0.590,0.026,0.436,0.564,0.667,0.590,0.744,0.974,0.462
average_M5,0.846,0.641,0.051,0.436,0.590,0.667,0.590,0.744,0.974,0.615
average_position,1.545,1.760,2.500,1.529,1.391,1.308,1.043,1.178,1.184,2.704
average_length (x of gs),1.532,1.733,0.967,0.980,1.157,0.972,0.479,0.751,0.802,2.460


In [61]:
displayResults({key:val for key, val in results.items() if 'any' in key})

Results for experiment ignore_common_any_session_failed, which has 26 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.569,0.351,0.042,0.340,0.509,0.569,0.853,0.722,0.743,0.293
average_recall,0.713,0.505,0.046,0.306,0.483,0.487,0.363,0.475,0.576,0.598
average_f1-score,0.633,0.414,0.044,0.322,0.496,0.525,0.509,0.561,0.649,0.393
average_NDCG,0.755,0.533,0.048,0.414,0.644,0.579,0.605,0.659,0.794,0.540
average_M1,0.577,0.346,0.038,0.385,0.615,0.500,0.654,0.654,0.808,0.308
average_M3,0.885,0.615,0.038,0.577,0.769,0.731,0.692,0.808,1.000,0.538
average_M5,0.923,0.692,0.077,0.577,0.808,0.731,0.692,0.808,1.000,0.692
average_position,1.500,1.889,2.500,1.533,1.429,1.368,1.056,1.205,1.192,2.421
average_length (x of gs),1.464,1.785,0.967,0.987,1.114,1.047,0.487,0.797,0.856,2.382


Results for experiment ignore_common_any_no_disease_found, which has 3 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.889,0.500,0.361,0.167,0.500,0.389,1.000,0.722,0.778,0.143
average_recall,0.689,0.400,0.400,0.067,0.244,0.356,0.422,0.430,0.511,0.200
average_f1-score,0.776,0.444,0.380,0.096,0.328,0.372,0.594,0.528,0.617,0.167
average_NDCG,0.757,0.662,0.418,0.226,0.507,0.181,0.810,0.616,0.858,0.203
average_M1,0.667,0.667,0.333,0.333,0.667,0.000,1.000,0.667,1.000,0.000
average_M3,1.000,1.000,0.333,0.333,0.667,0.000,1.000,0.667,1.000,0.000
average_M5,1.000,1.000,0.667,0.333,0.667,0.000,1.000,0.667,1.000,0.000
average_position,1.333,1.333,2.500,1.000,1.000,0.000,1.000,0.667,1.000,7.000
average_length (x of gs),0.800,1.178,0.967,0.578,0.533,0.933,0.422,0.666,0.644,1.467


Results for experiment ignore_less common_any_session_failed, which has 7 cases, is


...

,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average_recall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average_f1-score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
average_NDCG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average_M1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average_M3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average_M5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average_position,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average_length (x of gs),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Results for experiment ignore_all_any_session_failed, which has 33 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.499,0.327,0.033,0.298,0.435,0.571,0.778,0.690,0.722,0.269
average_recall,0.643,0.484,0.036,0.272,0.421,0.449,0.341,0.450,0.560,0.552
average_f1-score,0.562,0.390,0.034,0.284,0.428,0.503,0.474,0.536,0.631,0.362
average_NDCG,0.650,0.488,0.038,0.345,0.521,0.530,0.507,0.599,0.760,0.484
average_M1,0.455,0.303,0.030,0.303,0.485,0.455,0.515,0.576,0.758,0.273
average_M3,0.788,0.576,0.030,0.485,0.606,0.667,0.545,0.727,0.970,0.455
average_M5,0.818,0.636,0.061,0.485,0.636,0.667,0.545,0.727,0.970,0.606
average_position,1.593,1.905,2.500,1.562,1.429,1.364,1.056,1.213,1.219,2.636
average_length (x of gs),1.522,1.811,0.967,0.997,1.136,0.992,0.490,0.781,0.862,2.367


Results for experiment ignore_all_any_no_disease_found, which has 3 cases, is


,Ada,Avey,Babylon,Buoy,K health,Mohmmad Almadani,Noor Joudeh,average_doctor,Tala Hammouri,WebMD
average_precision,0.889,0.500,0.361,0.167,0.500,0.389,1.000,0.722,0.778,0.143
average_recall,0.689,0.400,0.400,0.067,0.244,0.356,0.422,0.430,0.511,0.200
average_f1-score,0.776,0.444,0.380,0.096,0.328,0.372,0.594,0.528,0.617,0.167
average_NDCG,0.757,0.662,0.418,0.226,0.507,0.181,0.810,0.616,0.858,0.203
average_M1,0.667,0.667,0.333,0.333,0.667,0.000,1.000,0.667,1.000,0.000
average_M3,1.000,1.000,0.333,0.333,0.667,0.000,1.000,0.667,1.000,0.000
average_M5,1.000,1.000,0.667,0.333,0.667,0.000,1.000,0.667,1.000,0.000
average_position,1.333,1.333,2.500,1.000,1.000,0.000,1.000,0.667,1.000,7.000
average_length (x of gs),0.800,1.178,0.967,0.578,0.533,0.933,0.422,0.666,0.644,1.467


Now let us combine the individual results of the apps.

In [62]:
combinedResults = {}
for label, keys in combineLabels.items():
    collectedResults = {}
    for key in keys:
        result = results[key]
        app = key.split('_')[2]
        collectedResults[app] = result.loc[:,app]
    combinedResults[label] = pd.DataFrame(
        collectedResults,
        index=[
            f"average_{x}" for x in [
                "precision", "recall", "f1-score", "NDCG", "M1", "M3", "M5", "position", "length (x of gs)"
            ]
        ]
        )

In [63]:
displayResults(combinedResults,printNumCases=False)

Results for experiment ignore_common_none is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.538,0.369,0.034,0.312,0.460,0.272
average_recall,0.694,0.515,0.038,0.291,0.455,0.595
average_f1-score,0.606,0.430,0.036,0.301,0.457,0.373
average_NDCG,0.752,0.551,0.039,0.370,0.586,0.527
average_M1,0.594,0.406,0.031,0.344,0.562,0.281
average_M3,0.906,0.625,0.031,0.500,0.688,0.531
average_M5,0.938,0.688,0.062,0.500,0.719,0.688
average_position,1.467,1.727,2.500,1.500,1.391,2.542
average_length (x of gs),1.486,1.695,0.967,0.967,1.143,2.493


Results for experiment ignore_common_session_failed is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.538,0.369,0.035,0.345,0.526,0.272
average_recall,0.694,0.515,0.039,0.321,0.520,0.595
average_f1-score,0.606,0.430,0.037,0.333,0.523,0.373
average_NDCG,0.752,0.551,0.040,0.408,0.669,0.527
average_M1,0.594,0.406,0.032,0.379,0.643,0.281
average_M3,0.906,0.625,0.032,0.552,0.786,0.531
average_M5,0.938,0.688,0.065,0.552,0.821,0.688
average_position,1.467,1.727,2.500,1.500,1.391,2.542
average_length (x of gs),1.486,1.695,0.967,0.967,1.143,2.493


Results for experiment ignore_common_no_disease_found is


...

,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.485,0.345,0.028,0.282,0.406,0.256
average_recall,0.638,0.495,0.031,0.264,0.408,0.557
average_f1-score,0.551,0.407,0.029,0.273,0.407,0.351
average_NDCG,0.663,0.510,0.032,0.319,0.492,0.482
average_M1,0.487,0.359,0.026,0.282,0.462,0.256
average_M3,0.821,0.590,0.026,0.436,0.564,0.462
average_M5,0.846,0.641,0.051,0.436,0.590,0.615
average_position,1.545,1.760,2.500,1.529,1.391,2.704
average_length (x of gs),1.532,1.733,0.967,0.980,1.157,2.460


Results for experiment ignore_all_session_failed is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.485,0.345,0.029,0.306,0.453,0.256
average_recall,0.638,0.495,0.032,0.286,0.454,0.557
average_f1-score,0.551,0.407,0.030,0.296,0.453,0.351
average_NDCG,0.663,0.510,0.033,0.346,0.548,0.482
average_M1,0.487,0.359,0.026,0.306,0.514,0.256
average_M3,0.821,0.590,0.026,0.472,0.629,0.462
average_M5,0.846,0.641,0.053,0.472,0.657,0.615
average_position,1.545,1.760,2.500,1.529,1.391,2.704
average_length (x of gs),1.532,1.733,0.967,0.980,1.157,2.460


Results for experiment ignore_all_no_disease_found is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.485,0.345,0.361,0.344,0.406,0.256
average_recall,0.638,0.495,0.400,0.322,0.408,0.557
average_f1-score,0.551,0.407,0.380,0.333,0.407,0.351
average_NDCG,0.663,0.510,0.418,0.389,0.492,0.482
average_M1,0.487,0.359,0.333,0.344,0.462,0.256
average_M3,0.821,0.590,0.333,0.531,0.564,0.462
average_M5,0.846,0.641,0.667,0.531,0.590,0.615
average_position,1.545,1.760,2.500,1.529,1.391,2.704
average_length (x of gs),1.532,1.733,0.967,0.980,1.157,2.460
